# Main script to Join Data on Instrument 2 Fire Location

Modules: N/A <br>
Author: Jordan Meyer <br>
Email: jordan.meyer@berkeley.edu <br>
Date created: Feb 18, 2023 <br>

**Citations (data sources)**


**Citations (persons)**
1. Cornelia Ilin 

**Preferred environment**
1. Code written in Jupyter Notebooks

### Step 1: Import packages

In [2]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install geopandas --quiet
    from google.colab import drive

    drive.mount("/content/drive")
    in_dir = (
        in_instrument
    ) = "/content/drive/MyDrive/capstone_fire/notebooks/instrument_2/"
    import drive.MyDrive.capstone_fire.modules.deep_ols as deep_ols
else:
    in_dir = in_instrument = "../data/instrument_2/"
    %cd '..'
    import modules.deep_ols as deep_ols
    %cd 'notebooks'

/Users/jordan/Documents/GitHub/fire_capstone
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [3]:
import calendar
import os
from datetime import date, timedelta

# geography
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import shapely

# Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
from shapely.geometry import Point

# import cartopy.crs as ccrs
# import contextily as ctx
# import fiona
# import netCDF4 as ncdf
# import osmnx as ox
# from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
# from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


dist = sklearn.metrics.DistanceMetric.get_metric("haversine")

# ignore warnings
import warnings

from tqdm.notebook import tqdm, trange

warnings.filterwarnings("ignore")

### Step 2: Define working directories

### Step 3: Define functions

### Step 4: Read data

## Wind

Wind load from other file

In [4]:
wind_df = (
    (
        pd.read_csv(in_dir + "all_years_wind_data.csv", index_col=0)
        .sort_values(["year_month", "ZCTA10"])
        .reset_index(drop=True)
    )
    .drop("Unnamed: 0", axis=1)
    .rename(columns={"ZCTA10": "ZCTA"})
    .reset_index(drop=True)
    .drop_duplicates()
)


wind_df["year_month"] = wind_df["year_month"].astype(str)
wind_df["ZCTA"] = wind_df["ZCTA"].astype(str)


wind_df[wind_df.ZCTA == "97635"].lat = 41.993
# wind_ready.loc['97635','ZCTA' == '97635']#['lon'] = -120.250
# wind_ready = wind_ready.drop_duplicates()
ind_fix = wind_df[wind_df.ZCTA == "97635"].index
for ind in ind_fix:
    wind_df.iloc[ind, 0] = 41.993
    wind_df.iloc[ind, 1] = -120.250

wind_dates = wind_df["year_month"].unique()
wind_zips = wind_df["ZCTA"].unique()
wind_ready = wind_df
wind_ready

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,year,month
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,1991,1
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,1991,1
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,1991,1
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,1991,1
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,1991,1
...,...,...,...,...,...,...,...,...,...,...
627067,39.149,-120.248,96146,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627068,39.236,-120.062,96148,0.636599,0.953782,56.279018,1.146716,202212,2022,12
627069,38.732,-120.033,96150,0.071517,0.472164,81.387100,0.477549,202212,2022,12
627070,39.184,-120.427,96161,-0.217010,0.239125,132.224274,0.322915,202212,2022,12


## Fire

Finsh cleaning some fire things and then join fire to wind 

In [5]:
fire_df = pd.read_csv(in_instrument + "fire_wind_processed.csv", index_col=0)

# Extract year and month, filter for fires in scope 1991+
# fire_df["year"] = fire_df["ALARM_DATE"].str[:4]
fire_df = fire_df[fire_df["year"].astype(int) > 1990]
fire_df = fire_df[fire_df["year"].astype(int) < 2020]

# Extract Month and End Year/Month
fire_df["month"] = fire_df["month"].astype(int)
fire_df["end_year"] = fire_df["CONT_DATE"].str[:4]
fire_df["end_month"] = fire_df["CONT_DATE"].str[5:7].astype(int)

# Give each fire a unique index code for later
fire_df = (
    fire_df.drop_duplicates().reset_index().rename(columns={"Index": "fire_index"})
)

fire_df["geometry"] = gpd.GeoSeries.from_wkt(fire_df["fire_centroid"])
fire_df["lon"] = fire_df["geometry"].apply(lambda p: p.x)
fire_df["lat"] = fire_df["geometry"].apply(lambda p: p.y)

fire_df["sq_mi"] = fire_df["GIS_ACRES"] / 640
fire_df["radius_mi"] = np.sqrt(fire_df["sq_mi"] / np.pi)

# Give each fire a unique index code for later
fire_df = (
    fire_df.drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "fire_index"})
)

fire_dates = fire_df["year_month"].unique()
fire_indexes = fire_df.index

fire_df.year = fire_df.year.astype(int)
fire_df.month = fire_df.month.astype(int)
fire_df["max_days_in_mo"] = fire_df[["year", "month"]].apply(
    lambda x: calendar.monthrange(*x)[1], axis=1
)

fire_df["sq_mi"] = fire_df["GIS_ACRES"] / 640
fire_df["radius_mi"] = np.sqrt(fire_df["sq_mi"] / np.pi)

fire_df["start_month"] = fire_df["ALARM_DATE"].apply(lambda x: int(x[5:7]))
fire_df["start_day"] = fire_df["ALARM_DATE"].apply(lambda x: int(x[8:10]))

fire_df["end_month"] = fire_df["CONT_DATE"].apply(lambda x: int(x[5:7]))
fire_df["end_day"] = fire_df["CONT_DATE"].apply(lambda x: int(x[8:10]))


fire_df["start_mid_end"] = fire_df[["month", "start_month", "end_month"]].apply(
    lambda x: (
        "complete"
        if x[1] == x[2]
        else ("start" if x[0] == x[1] else ("end" if x[0] == x[2] else "mid"))
    ),
    axis=1,
)

fire_df["fire_days_in_mo"] = fire_df[
    [
        "start_month",
        "end_month",
        "max_days_in_mo",
        "start_day",
        "end_day",
        "start_mid_end",
    ]
].apply(
    lambda x: (
        1
        if (x["end_day"] - x["start_day"] == 0)
        else (
            x["end_day"] - x["start_day"]
            if (x["start_mid_end"] == "complete")
            else (
                x["max_days_in_mo"] - x["start_day"]
                if (x["start_mid_end"] == "start")
                else (
                    x["end_day"]
                    if x["start_mid_end"] == "end"
                    else (x["max_days_in_mo"])
                )
            )
        )
    )
    / x["max_days_in_mo"],
    axis=1,
)

fire_backup = fire_df

fire_ready = fire_df[
    [
        "fire_lat_lon",
        "GIS_ACRES",
        "lat",
        "lon",
        "fire_days_in_mo",
        "fire_wdir",
        "fire_wspd",
        "year_month",
        "year",
        "month",
    ]
]
fire_ready = fire_ready.reset_index(drop=True).rename(
    columns={"GIS_ACRES": "acres", "lat": "fire_lat", "lon": "fire_lon"}
)

fire_ready

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8
...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12


## PM2.5

Load PM2.5 from lawis files downloaded from gdrive

In [6]:
aqi1 = pd.read_csv(in_instrument + "jordan_firepm25.csv", index_col=0)
aqi2 = pd.read_csv(in_instrument + "finalpm25.csv", index_col=0)
aqi2

,year_month,ZIP10,pm25
204,2017-01,89010,1.842857
205,2017-02,89010,3.528571
206,2017-03,89010,3.242857
207,2017-04,89010,3.700000
208,2017-05,89010,5.242857
...,...,...,...
585307,2016-12,96148,1.713333
585308,2016-12,96150,1.719288
585309,2016-12,96155,0.771287
585310,2016-12,96161,1.416996


In [7]:
pm25_df = aqi2.reset_index(drop=True).rename(
    columns={"ZIP10": "ZCTA", "year_month": "old_ym"}
)
pm25_df["year_month"] = pm25_df["old_ym"].str[:4] + pm25_df["old_ym"].str[5:]
pm25_df["ZCTA"] = pm25_df["ZCTA"].astype(str)
pm25_ready = pm25_df.drop("old_ym", axis=1)

In [8]:
wfp_df = wind_ready.merge(pm25_ready, on=["year_month", "ZCTA"], how="left")
wind_final = wfp_df.rename(
    columns={
        "lat": "zip_lat",
        "lon": "zip_lon",
        "wdir": "zip_wdir",
        "wspd": "zip_wspd",
        "pm25": "zip_pm25",
    }
).drop(["u", "v"], axis=1)[wfp_df["year"] < 2019]
wind_final.to_csv(in_instrument + "wind_zip_pm25.csv")
wind_final

,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year,month,zip_pm25
0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976
1,35.396,-116.322,89019,259.661102,0.962568,199101,1991,1,10.846541
2,36.161,-116.139,89060,241.796738,0.922477,199101,1991,1,12.385000
3,35.957,-115.897,89061,244.522552,1.303105,199101,1991,1,10.150000
4,39.520,-120.032,89439,78.390099,0.209957,199101,1991,1,8.389565
...,...,...,...,...,...,...,...,...,...
548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316
548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572
548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667
548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509


In [9]:
fire_pm25 = aqi1.rename(
    columns={"lat": "fire_lat", "lon": "fire_lon", "pm25": "fire_pm25"}
).reset_index(drop=True)
fire_pm25.to_csv("fire_pm25.csv")
fire_pm25

,year,month,fire_lat,fire_lon,fire_pm25
0,1991,1,39.155351,-122.775965,0.10
1,1991,1,39.567560,-121.002336,11.24
2,1991,1,39.084997,-122.744482,1.74
3,1991,1,39.756806,-121.331523,10.59
4,1991,1,39.694403,-121.355888,15.02
...,...,...,...,...,...
8267,2016,11,40.375938,-120.751081,3.61
8268,2016,12,34.047386,-118.948054,4.52
8269,2016,12,34.234554,-118.800483,5.81
8270,2016,12,34.122593,-118.722263,6.09


In [16]:
fire_final = (
    fire_ready.merge(
        fire_pm25, on=["fire_lat", "fire_lon", "year", "month"], how="left"
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
fire_final

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,fire_pm25
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,9.38
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,5.53
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,14.24
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,11.99
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,9.19
...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,4.76
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,4.38
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,1.70
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,3.80


In [89]:
wind_final[wind_final.year == 2001]

,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year,month,zip_pm25,wind_lat_lon
195960,37.465,-117.936,89010,327.503998,0.498294,200101,2001,1,5.580488,"(37.465, -117.936)"
195961,35.396,-116.322,89019,271.946747,0.505702,200101,2001,1,5.101090,"(35.396, -116.322)"
195962,36.161,-116.139,89060,235.695648,0.547247,200101,2001,1,4.714583,"(36.161, -116.139)"
195963,35.957,-115.897,89061,240.307678,0.877861,200101,2001,1,5.290000,"(35.957, -115.897)"
195964,39.520,-120.032,89439,109.936600,0.416530,200101,2001,1,4.554783,"(39.52, -120.032)"
...,...,...,...,...,...,...,...,...,...,...
215551,39.149,-120.248,96146,56.552437,0.879490,200112,2001,12,0.940893,"(39.149, -120.248)"
215552,39.236,-120.062,96148,56.552437,0.879490,200112,2001,12,1.383333,"(39.236, -120.062)"
215553,38.732,-120.033,96150,77.229973,0.385849,200112,2001,12,1.315252,"(38.732, -120.033)"
215554,39.184,-120.427,96161,131.936325,0.309035,200112,2001,12,1.524918,"(39.184, -120.427)"


In [87]:
def distance(wind, fire):
    """
    params:
    -------
    wind: wind cleaned data
    fire: fire semi-cleaned data

    return:
    -------
    df3: dataframe with distances from each fire during time period
    """
    # create labels
    wind["wind_lat_lon"] = [str(xy) for xy in zip(wind.zip_lat, wind.zip_lon)]
    fire["fire_lat_lon"] = [str(xy) for xy in zip(fire.fire_lat, fire.fire_lon)]

    ## for each point in wind data find the nearest point in the census data ##
    ###############
    # keep only unique points in fire data
    df1_unique = wind.drop_duplicates(["wind_lat_lon"])
    df2_unique = fire.drop_duplicates(["fire_lat_lon"])

    df1_unique.reset_index(drop=True, inplace=True)
    df2_unique.reset_index(drop=True, inplace=True)

    # transform to radians
    df1_unique["lat_r"] = np.radians(df1_unique.zip_lat)
    df1_unique["lon_r"] = np.radians(df1_unique.zip_lon)
    df2_unique["lat_r"] = np.radians(df2_unique.fire_lat)
    df2_unique["lon_r"] = np.radians(df2_unique.fire_lon)

    # compute pairwise distance (in miles)
    dist_matrix = (
        dist.pairwise(
            df2_unique[["lat_r", "lon_r"]],
            df1_unique[["lat_r", "lon_r"]],
        )
    ) * 3959

    # create a df from dist_matrix
    dist_matrix = pd.DataFrame(
        dist_matrix, index=df2_unique["fire_lat_lon"], columns=df1_unique["ZCTA"]
    )
    return dist_matrix

In [91]:
temp_df = distance(wind_final, fire_final)
dist_df = (
    fire_final.merge(
        temp_df, left_on="fire_lat_lon", right_on=temp_df.index, how="left"
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
dist_df[dist_df['year']==2001]

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
4263,"(38.99382291250062, -119.8900734265666)",14.999589,38.993823,-119.890073,0.066667,26.465803,1.353703,200106,2001,6,...,17.982436,15.988476,14.666087,17.027272,17.789304,21.991831,19.104429,19.657816,31.652662,208.096525
4264,"(37.18793272219135, -119.3016027227447)",210.065674,37.187933,-119.301603,0.129032,351.406494,0.150014,200108,2001,8,...,145.466788,136.085877,128.852747,145.715757,138.521533,144.927855,147.413404,113.887952,150.856280,335.830309
4265,"(37.77325378939722, -120.2357674605692)",11095.119141,37.773254,-120.235767,0.193548,70.758270,0.115202,200108,2001,8,...,98.728079,86.952417,79.994381,100.846529,88.991087,95.063045,101.507973,67.154431,98.026390,291.575248
4266,"(39.78646240712992, -122.7220776684899)",437.361511,39.786462,-122.722078,0.032258,304.883057,1.167287,200105,2001,5,...,145.706338,145.201480,149.637633,149.753193,143.141519,139.123241,146.811354,161.257553,129.274526,199.782329
4267,"(38.19506558007031, -120.3070265610761)",14280.477539,38.195066,-120.307027,0.633333,106.467476,0.052246,200109,2001,9,...,70.211091,58.141704,51.417849,72.743463,60.062757,65.991431,73.129049,39.953419,68.638634,262.445388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4362,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.032258,355.636200,0.409346,200110,2001,10,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453
4363,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.033333,35.386692,0.565489,200111,2001,11,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453
4364,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.032258,22.306015,0.657902,200112,2001,12,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453
4365,"(36.41759572693118, -118.6730547651234)",508.639557,36.417596,-118.673055,1.000000,200.557068,0.217325,200110,2001,10,...,207.262473,198.557065,191.402907,207.055488,201.026137,207.397163,208.970696,176.405167,213.784570,394.369475


In [20]:
def fire_bearings(wind, fire):
    """
    params:
    -------
    wind: wind cleaned data
    fire: fire semi-cleaned data

    return:
    -------
    df3: dataframe with distances from each fire during time period
    """

    ## for each point in wind data find the angle##
    ###############
    # keep only unique points in fire data
    df1_unique = wind.drop_duplicates(["wind_lat_lon"])
    df2_unique = fire.drop_duplicates(["fire_lat_lon"])

    df1_unique.reset_index(drop=True, inplace=True)
    df2_unique.reset_index(drop=True, inplace=True)

    # transform to radians
    df1_unique["lat_r"] = np.radians(df1_unique.zip_lat)
    df1_unique["lon_r"] = np.radians(df1_unique.zip_lon)
    df2_unique["lat_r"] = np.radians(df2_unique.fire_lat)
    df2_unique["lon_r"] = np.radians(df2_unique.fire_lon)

    # compute pairwise distance (in miles)
    bearing_matrix = np.zeros((df1_unique.shape[0], df2_unique.shape[0]))
    for idx in range(df1_unique.shape[0]):
        pa_lat = df1_unique["zip_lat"][idx]
        pa_lon = df1_unique["zip_lon"][idx]
        wdir_ba = df2_unique["fire_wdir"][idx]
        for idy in range(df2_unique.shape[0]):
            pb_lat = df2_unique["fire_lat"][idy]
            pb_lon = df2_unique["fire_lon"][idy]

            if pb_lon > pa_lon:
                d_X = -np.cos(pa_lat) * np.sin(pa_lon - pb_lon)
            else:
                d_X = np.cos(pa_lat) * np.sin(pa_lon - pb_lon)

            if pb_lat > pa_lat:
                d_Y = -np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(
                    pa_lat
                ) * np.cos(pa_lon - pb_lon)
            else:
                d_Y = np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(
                    pa_lat
                ) * np.cos(pa_lon - pb_lon)

            B_ba = np.arctan2(d_X, d_Y) * (180 / np.pi) % 360

            angle_diff = np.max([B_ba, wdir_ba]) - np.min([B_ba, wdir_ba])

            if angle_diff > 180:
                upwind_effect = (angle_diff) / 360
            else:
                upwind_effect = 1 - (angle_diff) / 360

            bearing_matrix[idx, idy] = upwind_effect

    # create a df from dist_matrix
    bearing_matrix = pd.DataFrame(
        bearing_matrix, index=df1_unique["ZCTA"], columns=df2_unique["fire_lat_lon"]
    )

    return bearing_matrix

In [21]:
temp_df = fire_bearings(wind_final, fire_final)

In [22]:
temp_df = temp_df.fillna(0)
merge_temp_df = temp_df.T
bearings = fire_ready.merge(
    merge_temp_df, left_on="fire_lat_lon", right_on=merge_temp_df.index, how="left"
)
bearings_fire = bearings

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
4263,"(38.99382291250062, -119.8900734265666)",14.999589,38.993823,-119.890073,0.066667,26.465803,1.353703,200106,2001,6,...,0.950738,0.713314,0.635292,0.958817,0.980532,0.920267,0.890424,0.593172,0.935264,0.759977
4264,"(37.18793272219135, -119.3016027227447)",210.065674,37.187933,-119.301603,0.129032,351.406494,0.150014,200108,2001,8,...,0.953146,0.836597,0.930537,0.957265,0.936001,0.896869,0.890791,0.834986,0.914850,0.543768
4265,"(37.77325378939722, -120.2357674605692)",11095.119141,37.773254,-120.235767,0.193548,70.758270,0.115202,200108,2001,8,...,0.960687,0.865726,0.893593,0.961122,0.908757,0.879877,0.887522,0.870856,0.903621,0.597963
4266,"(39.78646240712992, -122.7220776684899)",437.361511,39.786462,-122.722078,0.032258,304.883057,1.167287,200105,2001,5,...,0.951945,0.835311,0.928175,0.956869,0.938392,0.900272,0.891390,0.848588,0.919792,0.747186
4267,"(38.19506558007031, -120.3070265610761)",14280.477539,38.195066,-120.307027,0.633333,106.467476,0.052246,200109,2001,9,...,0.959463,0.861555,0.900977,0.960238,0.912066,0.881025,0.888100,0.849371,0.902948,0.592973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4362,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.032258,355.636200,0.409346,200110,2001,10,...,0.946704,0.816486,0.959528,0.952629,0.953204,0.906255,0.893921,0.790423,0.919844,0.510419
4363,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.033333,35.386692,0.565489,200111,2001,11,...,0.946704,0.816486,0.959528,0.952629,0.953204,0.906255,0.893921,0.790423,0.919844,0.510419
4364,"(36.93386873267413, -118.6577934353378)",9.361344,36.933869,-118.657793,0.032258,22.306015,0.657902,200112,2001,12,...,0.946704,0.816486,0.959528,0.952629,0.953204,0.906255,0.893921,0.790423,0.919844,0.510419
4365,"(36.41759572693118, -118.6730547651234)",508.639557,36.417596,-118.673055,1.000000,200.557068,0.217325,200110,2001,10,...,0.924584,0.759179,0.977830,0.937961,0.992261,0.943819,0.904742,0.737366,0.949323,0.560895


In [ ]:
# treatment_df = (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#     dist_df.drop_duplicates().iloc[:, 11:] < 200
# ].fillna(False)

# treatment_df["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
# treatment_df["year_month"] = bearings_fire.drop_duplicates()["year_month"]
# treatment_df["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
# treatment_df["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

# treatment_df["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
# treatment_df = treatment_df.reset_index(drop=True)

In [45]:
fire_bear = bearings_fire
# [
#     (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#         dist_df.drop_duplicates().iloc[:, 11:] < 200
#     ]
# ].fillna(0)

fire_bear["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
fire_bear["year_month"] = bearings_fire.drop_duplicates()["year_month"]
fire_bear["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
fire_bear["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

fire_bear["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
fire_bear = fire_bear.reset_index(drop=True)

fire_bear.to_csv("fire_bear.csv")

In [46]:
fire_dist = dist_df
# [
#     (bearings_fire.drop_duplicates().iloc[:, 11:] >= 0.75)[
#         dist_df.drop_duplicates().iloc[:, 11:] < 200
#     ]
# ].fillna(0)

fire_dist["fire_lat_lon"] = bearings.drop_duplicates()["fire_lat_lon"].astype(str)
fire_dist["year_month"] = bearings_fire.drop_duplicates()["year_month"]
fire_dist["fire_wdir"] = bearings_fire.drop_duplicates()["fire_wdir"]
fire_dist["fire_wspd"] = bearings_fire.drop_duplicates()["fire_wspd"]

fire_dist["fire_days_in_mo"] = bearings_fire.drop_duplicates()["fire_days_in_mo"]
fire_dist = fire_dist.reset_index(drop=True)

fire_dist.to_csv("fire_dist.csv")

In [26]:
days_mo = fire_dist["fire_days_in_mo"][2]
wspd = fire_dist["fire_wspd"][2]
distance = fire_dist["96142"][2]
bearing = fire_bear["96142"][2]

instrument = np.sum(days_mo * (wspd / distance) * bearing)
instrument

0.0002073982392657109

In [97]:
fire_dist = pd.read_csv("fire_dist.csv", index_col=1)
fire_bear = pd.read_csv("fire_bear.csv", index_col=1)

In [99]:
fire_dist[fire_dist.year == 2001]

,Unnamed: 0,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
fire_lat_lon,,,,,,,,,,,,,,,,,,,,,
"(38.99382291250062, -119.8900734265666)",4263,14.999589,38.993823,-119.890073,0.066667,26.465803,1.353703,200106,2001,6,...,17.982436,15.988476,14.666087,17.027272,17.789304,21.991831,19.104429,19.657816,31.652662,208.096525
"(37.18793272219135, -119.3016027227447)",4264,210.065674,37.187933,-119.301603,0.129032,351.406494,0.150014,200108,2001,8,...,145.466788,136.085877,128.852747,145.715757,138.521533,144.927855,147.413404,113.887952,150.856280,335.830309
"(37.77325378939722, -120.2357674605692)",4265,11095.119141,37.773254,-120.235767,0.193548,70.758270,0.115202,200108,2001,8,...,98.728079,86.952417,79.994381,100.846529,88.991087,95.063045,101.507973,67.154431,98.026390,291.575248
"(39.78646240712992, -122.7220776684899)",4266,437.361511,39.786462,-122.722078,0.032258,304.883057,1.167287,200105,2001,5,...,145.706338,145.201480,149.637633,149.753193,143.141519,139.123241,146.811354,161.257553,129.274526,199.782329
"(38.19506558007031, -120.3070265610761)",4267,14280.477539,38.195066,-120.307027,0.633333,106.467476,0.052246,200109,2001,9,...,70.211091,58.141704,51.417849,72.743463,60.062757,65.991431,73.129049,39.953419,68.638634,262.445388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(36.93386873267413, -118.6577934353378)",4362,9.361344,36.933869,-118.657793,0.032258,355.636200,0.409346,200110,2001,10,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453
"(36.93386873267413, -118.6577934353378)",4363,9.361344,36.933869,-118.657793,0.033333,35.386692,0.565489,200111,2001,11,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453
"(36.93386873267413, -118.6577934353378)",4364,9.361344,36.933869,-118.657793,0.032258,22.306015,0.657902,200112,2001,12,...,174.961128,167.047200,160.052895,174.368226,169.541034,175.821770,176.447812,145.149119,182.852422,359.724453


In [ ]:
import multiprocessing

from tqdm import tqdm

iters = int(np.ceil(fire_dist.shape[0] / 100))
instrument_df = pd.DataFrame(
    columns=[
        "ZCTA",
        "fire_lat_lon",
        "instrument",
        "instrument_norm",
        "year_month",
        "bearing",
        "distance",
        "fire_wspd",
    ]
)

In [42]:
fire_dist

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,...,273.902165,262.986709,255.763437,275.094422,265.225259,271.526768,276.316781,241.513929,275.412483,466.944962
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,...,18.155774,30.531542,36.890962,15.868377,29.049318,24.497956,15.164144,49.742314,28.990727,176.052420
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,...,55.923517,44.852443,42.502675,60.493373,45.091172,47.913347,59.138386,44.100498,44.201396,234.375215
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,...,261.622226,250.503922,243.318349,262.985454,252.701931,258.958683,264.112075,229.258884,262.543176,454.860185
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,...,283.319934,274.116537,266.888233,283.351792,276.555557,282.960181,285.162251,251.917314,288.832200,471.414260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,...,93.579473,83.601955,76.330517,94.315845,85.999136,92.411551,95.759703,61.483932,98.022876,285.885768
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,...,115.058673,104.965884,97.687268,115.806581,107.344223,113.752581,117.250544,82.902744,119.099573,307.313468
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,...,102.401609,92.355699,85.079677,103.152564,94.742784,101.153523,104.591706,70.267223,106.628801,294.707998
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,...,102.401609,92.355699,85.079677,103.152564,94.742784,101.153523,104.591706,70.267223,106.628801,294.707998


In [43]:
fire_bear

,fire_lat_lon,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd,year_month,year,month,...,96140,96141,96142,96143,96145,96146,96148,96150,96161,97635
0,"(35.25539508507167, -119.5812970773511)",238.031830,35.255395,-119.581297,0.032258,342.554565,1.595909,199305,1993,5,...,0.529931,0.598269,0.662514,0.532863,0.622019,0.637264,0.616758,0.521091,0.613033,0.826084
1,"(39.45165114943642, -120.0093608630314)",19.139362,39.451651,-120.009361,0.032258,0.700786,1.000503,199307,1993,7,...,0.953073,0.528072,0.658048,0.962805,0.761105,0.946191,0.888510,0.603677,0.952879,0.873414
2,"(38.63102591471577, -120.8402934705149)",70.179840,38.631026,-120.840293,0.033333,58.361294,0.271346,199309,1993,9,...,0.948314,0.816085,0.974584,0.952978,0.950189,0.900464,0.893150,0.728573,0.911306,0.510884
3,"(35.42095813876129, -119.7656494555765)",23.423927,35.420958,-119.765649,0.033333,331.157288,1.023493,199309,1993,9,...,0.531475,0.599915,0.662499,0.534155,0.620580,0.636103,0.615938,0.507633,0.613045,0.816083
4,"(35.32190356232154, -118.4169237147862)",151.780167,35.321904,-118.416924,0.032258,7.345666,1.831764,199308,1993,8,...,0.522109,0.583436,0.679973,0.526616,0.634721,0.645291,0.620852,0.501348,0.615551,0.822699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,"(37.87494989081461, -119.7281350256987)",331.624695,37.874950,-119.728135,0.166667,73.448456,0.113857,200709,2007,9,...,0.958015,0.849817,0.913327,0.960668,0.924244,0.889567,0.888391,0.859021,0.910045,0.561256
6633,"(37.56860595616687, -119.6600328204804)",23.633701,37.568606,-119.660033,0.233333,158.267761,0.253292,200711,2007,11,...,0.957414,0.848830,0.914134,0.960218,0.925177,0.890298,0.888705,0.858714,0.910497,0.561818
6634,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,1.000000,215.334213,0.138538,200711,2007,11,...,0.957850,0.849720,0.913217,0.960531,0.924352,0.889704,0.888483,0.859562,0.910096,0.562145
6635,"(37.74854288820926, -119.7035432531625)",239.260773,37.748543,-119.703543,0.258065,178.254211,0.474964,200712,2007,12,...,0.957850,0.849720,0.913217,0.960531,0.924352,0.889704,0.888483,0.859562,0.910096,0.562145


In [ ]:
fire_dist.fire_wspd.dtype

dtype('float64')

In [ ]:
list(fire_dist.columns).index("fire_lat_lon")

0

In [106]:
fire_dist.columns[20:30]

Index(['90005', '90006', '90007', '90008', '90010', '90011', '90012', '90013',
       '90014', '90015'],
      dtype='object')

In [ ]:
def get_instrument(distance_df, bearing_df, instrument_df):
    # dist_max = np.max(np.max(distance_df.iloc[:, 11:]))
    # dist_min = 0
    # wspd_max = np.max(distance_df["fire_wspd"])
    # wspd_min = 0
    # acres_max = np.max(distance_df["acres"])
    # acres_min = np.min(distance_df["acres"])

    for ym in tqdm(range(distance_df.shape[0]), leave=False):
        wspd = distance_df.iloc[ym, 6]  # ["fire_wspd"][ym]
        for zcta in tqdm(range(11, distance_df.shape[1] - 1), leave=False):
            distance = distance_df.iloc[ym, zcta]
            bearing = bearing_df.iloc[ym, zcta]
            # instrument = days_mo * wspd * bearing / distance
            # dist_norm = (distance) / (dist_max)
            # wspd_norm = (wspd) / (wspd_max)
            # bear_norm = (bearing - 0.5) / (0.5)
            # instrument_norm = days_mo * wspd_norm * bear_norm / dist_norm

            instrument_df = instrument_df.append(
                {
                    "ZCTA": zcta,
                    "fire_lat_lon": distance_df.iloc[ym, 0],
                    "instrument": np.nan,
                    "year_month": distance_df.iloc[ym, 7],  # ["year_month"][ym],
                    "bearing": bearing,
                    "dist": distance,
                    "fire_wspd": wspd,
                    # "new_bearing": bear_norm,
                    # "wspd_scaled": wspd_norm,
                    # "dist_scaled": dist_norm,
                    # "instrument_norm": instrument_norm,
                },
                ignore_index=True,
            )
    return instrument_df

In [ ]:
fire_dist.shape[0]

6637

In [ ]:
def get_instrument_multi(iter_1, iter_2):
    instrument_df = pd.DataFrame(
        columns=[
            "ZCTA",
            "fire_lat_lon",
            "instrument",
            "year_month",
            "bearing",
            "new_bearing",
            "dist",
            # "instrument_norm",
            # "fire_wspd",
            # "wspd_scaled",
            # "dist_scaled",
        ]
    )

    distance_df = fire_dist.iloc[iter_1:iter_2, :]
    bearing_df = fire_bear.iloc[iter_1:iter_2, :]

    dist_max = np.max(np.max(distance_df.iloc[:, 11:]))
    dist_min = 0
    wspd_max = np.max(distance_df["fire_wspd"])
    wspd_min = 0
    acres_max = np.max(distance_df["acres"])
    acres_min = np.min(distance_df["acres"])

    for ym in tqdm(distance_df.index):
        days_mo = distance_df["fire_days_in_mo"][ym]
        wspd = distance_df["fire_wspd"][ym]
        for zcta in distance_df.columns[11:]:
            distance = distance_df[zcta][ym]
            bearing = bearing_df[zcta][ym]
            instrument = days_mo * wspd * bearing / distance

            #             dist_norm = (distance) / (dist_max)
            #             wspd_norm = (wspd) / (wspd_max)
            #             bear_norm = (bearing - 0.5) / (0.5)
            #             instrument_norm = days_mo * wspd_norm * bear_norm / dist_norm

            instrument_df = instrument_df.append(
                {
                    "ZCTA": zcta,
                    "fire_lat_lon": distance_df["fire_lat_lon"][ym],
                    "instrument": instrument,
                    "year_month": distance_df["year_month"][ym],
                    "bearing": bearing,
                    "dist": distance,
                    "fire_wspd": wspd,
                    # "new_bearing": bear_norm,
                    # "wspd_scaled": wspd_norm,
                    # "dist_scaled": dist_norm,
                    # "instrument_norm": instrument_norm,
                },
                ignore_index=True,
            )
    pbar.update(1)
    print(f"process {iteration} complete")
    instrument_df.to_csv(in_instrument + f"multi-{iter_1}-{iter_2}.csv")
    return f"{iter_1}-{iter_2}"

In [ ]:
from modules.instrument_multi import get_instrument_multi as gim

In [ ]:
%cd notebooks

[Errno 2] No such file or directory: 'notebooks'
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [ ]:
SIZE = 50
START = 0

iters = int(np.ceil(fire_dist.shape[0] / SIZE))
items = params = [(i * SIZE, i * SIZE + SIZE) for i in range(START, iters)]
tasks = range(iters)

In [ ]:
# import multiprocessing as mp

# # Step 1: Use multiprocessing.Pool() and specify number of cores to use (here I use 4).
# pool = mp.Pool(2)
# pbar = tqdm(total=len(tasks))

# with mp.Pool(4) as pool:
#     # Step 2: Use pool.starmap which takes a multiple iterable arguments
#     pool.starmap(gim, items)
#     # print(f'Got result: {result}', flush=True)

# pbar.close()

In [ ]:
SIZE = 50
START = 43

iters = int(np.ceil(fire_dist.shape[0] / SIZE))
items = params = [(i * SIZE, i * SIZE + SIZE) for i in range(START, iters)]
tasks = range(iters)

for i in tqdm(range(START, iters)):
    instrument_df = pd.DataFrame(
        columns=[
            "ZCTA",
            "fire_lat_lon",
            "instrument",
            "year_month",
            "bearing",
            "dist",
            "fire_wspd",
            # "wspd_scaled",
            # "dist_scaled",
            # "instrument_norm",
            # "new_bearing",
        ]
    )

    instrument_df = get_instrument(
        fire_dist[i * SIZE : i * SIZE + SIZE],
        fire_bear[i * SIZE : i * SIZE + SIZE],
        instrument_df,
    )
    instrument_df.to_csv(in_instrument + f"iter_{i}.csv")

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

...

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

In [ ]:
print("woo")

woo


## Consolidate iterations of lookups and merge for final df

In [48]:
import glob

files = glob.glob(in_instrument + "iter" + "*.csv")
df_all_instruments = pd.DataFrame()

total = 0
for f in files:
    total += 1
    csv = pd.read_csv(f, index_col=0)
    df_all_instruments = df_all_instruments.append(csv)
print(f"{total} files consoidated.")

df_all_instruments = df_all_instruments.reset_index(drop=True).drop_duplicates()
df_all_instruments.to_csv(in_instrument + "non_agg_instrument_2.csv")

133 files consoidated.


In [2]:
# instruments = df_all_instruments.reset_index(drop=True)


# instrument_final = (
#     instruments.groupby(["ZCTA", "year_month"]).aggregate(sum).drop_duplicates()
# )
# instrument_final.to_csv(in_instrument + "instrument_final.csv")

In [165]:
df_all_instruments= pd.read_csv(in_instrument + "non_agg_instrument_2.csv")

In [166]:
df_all_instruments[df_all_instruments['year_month']==200107]

,Unnamed: 0,ZCTA,fire_lat_lon,instrument,year_month,bearing,dist,fire_wspd
4908674,4908674,11,"(34.20609548544918, -117.3246819960429)",NaN,200107,0.902402,227.769615,1.625170
4908675,4908675,12,"(34.20609548544918, -117.3246819960429)",NaN,200107,0.843043,99.981554,1.625170
4908676,4908676,13,"(34.20609548544918, -117.3246819960429)",NaN,200107,0.948342,150.761219,1.625170
4908677,4908677,14,"(34.20609548544918, -117.3246819960429)",NaN,200107,0.503674,145.439436,1.625170
4908678,4908678,15,"(34.20609548544918, -117.3246819960429)",NaN,200107,0.952446,396.453940,1.625170
...,...,...,...,...,...,...,...,...
5925505,5925505,1638,"(34.17972305254299, -117.0517269002747)",NaN,200107,0.619168,379.982732,1.770589
5925506,5925506,1639,"(34.17972305254299, -117.0517269002747)",NaN,200107,0.614187,386.301965,1.770589
5925507,5925507,1640,"(34.17972305254299, -117.0517269002747)",NaN,200107,0.588049,387.066562,1.770589
5925508,5925508,1641,"(34.17972305254299, -117.0517269002747)",NaN,200107,0.595476,355.464104,1.770589


In [177]:
fire_dist.shape[1] - 1

1643

In [178]:
fire_dist.columns

Index(['Unnamed: 0', 'acres', 'fire_lat', 'fire_lon', 'fire_days_in_mo',
       'fire_wdir', 'fire_wspd', 'year_month', 'year', 'month',
       ...
       '96140', '96141', '96142', '96143', '96145', '96146', '96148', '96150',
       '96161', '97635'],
      dtype='object', length=1644)

In [3]:
# instruments = df_all_instruments.reset_index(drop=True)

In [117]:
wind_elevation = pd.read_csv("all_elevations.csv")
wind_elevation["zips"] = wind_elevation["zips"].astype(str)
wind_elevation["elevation"] = (
    wind_elevation["elevation"].map(lambda x: x[:-2]).astype(float)
)
wind_elevation = wind_final.merge(
    wind_elevation, left_on="ZCTA", right_on="zips", how="left"
)
wind_elevation = wind_elevation.rename(columns={"elevation": "zip_elevation"})

In [118]:
fire_elevation = pd.read_csv(in_instrument + "elevation_final.csv", index_col=0)
fire_elevation = fire_final.merge(
    fire_elevation, on=["fire_lat", "fire_lon"], how="left"
)

In [119]:
fire_elevation["year_month"] = fire_elevation["year_month"].astype(str)
df_all_instruments["ZCTA"] = df_all_instruments["ZCTA"].astype(str)
wind_elevation["ZCTA"] = wind_elevation["ZCTA"].astype(str)
df_all_instruments["year_month"] = df_all_instruments["year_month"].astype(str)

In [120]:
temp_instrument = df_all_instruments.drop_duplicates().merge(
    fire_elevation, on=["fire_lat_lon", "year_month"], how="left"
)

In [54]:
len(temp_instrument.year.unique())

26

In [59]:
sorted(temp_instrument.year.unique())

[1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

In [121]:
temp_wind_df = wind_elevation[
    ["ZCTA", "zip_elevation", "zip_pm25", "year_month"]
].drop_duplicates()

In [122]:
temp_wind_df['year']=temp_wind_df['year_month'].apply(lambda x: str(x)[:4])
temp_wind_df['month']=temp_wind_df['year_month'].apply(lambda x: str(x)[-2:])

In [180]:
temp_instrument['ZCTA'] = temp_instrument['ZCTA'].astype(int)

In [181]:
temp_fire_df = temp_instrument.rename(columns={"fire_wspd_x": "fire_wspd"})

In [195]:
temp_fire_df["real_ZCTA"] = temp_fire_df.ZCTA.map(lambda x: x if x > 2000 else fire_dist.columns[x])

In [196]:
temp_wind_df[temp_wind_df['year']=="2001"]

,ZCTA,zip_elevation,zip_pm25,year_month,year,month
195960,89010,4784.0,5.580488,200101,2001,01
195961,89019,4821.0,5.101090,200101,2001,01
195962,89060,2621.0,4.714583,200101,2001,01
195963,89061,2709.0,5.290000,200101,2001,01
195964,89439,5951.0,4.554783,200101,2001,01
...,...,...,...,...,...,...
215551,96146,6649.0,0.940893,200112,2001,12
215552,96148,6473.0,1.383333,200112,2001,12
215553,96150,6548.0,1.315252,200112,2001,12
215554,96161,6042.0,1.524918,200112,2001,12


In [200]:
temp_fire_df["real_ZCTA"] = temp_fire_df.ZCTA.map(lambda x: str(x) if x > 2000 else str(fire_dist.columns[x]))

In [201]:
temp_temp = temp_wind_df.merge(
    temp_fire_df, left_on=["ZCTA", "year_month"], right_on =["real_ZCTA","year_month"],how ='left'
).drop_duplicates()

In [202]:
temp_temp

,ZCTA_x,zip_elevation,zip_pm25,year_month,year_x,month_x,Unnamed: 0,ZCTA_y,fire_lat_lon,instrument,...,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd_y,year_y,month_y,fire_pm25,fire_elevation,real_ZCTA
0,89010,4784.0,12.450976,199101,1991,01,8391616.0,89010.0,"(39.15535115700015, -122.7759650805785)",0.000163,...,39.155351,-122.775965,0.064516,274.314545,0.929370,1991.0,1.0,0.10,1320.0,89010
1,89010,4784.0,12.450976,199101,1991,01,8409579.0,89010.0,"(39.56756030998135, -121.0023355975935)",0.000071,...,39.567560,-121.002336,0.032258,186.630615,0.516180,1991.0,1.0,11.24,1263.0,89010
2,89010,4784.0,12.450976,199101,1991,01,8414478.0,89010.0,"(39.08499688682003, -122.7444818242734)",0.000167,...,39.084997,-122.744482,0.064516,274.314545,0.929370,1991.0,1.0,1.74,1013.0,89010
3,89010,4784.0,12.450976,199101,1991,01,8417744.0,89010.0,"(39.75680610240665, -121.3315226354367)",0.000055,...,39.756806,-121.331523,0.096774,182.438004,0.138119,1991.0,1.0,10.59,1057.0,89010
4,89010,4784.0,12.450976,199101,1991,01,8419377.0,89010.0,"(39.69440333991243, -121.355888160909)",0.000207,...,39.694403,-121.355888,0.096774,186.630615,0.516180,1991.0,1.0,15.02,852.0,89010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19445497,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19445498,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19445499,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19445500,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
temp_temp[temp_temp['year_x']=="2001"]

,ZCTA_x,zip_elevation,zip_pm25,year_month,year_x,month_x,Unnamed: 0,ZCTA_y,fire_lat_lon,instrument,...,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd_y,year_y,month_y,fire_pm25,fire_elevation,real_ZCTA
2676395,89010,4784.0,5.580488,200101,2001,01,5879814.0,11.0,"(32.79739819490558, -116.734579970192)",NaN,...,32.797398,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89010
2676396,89010,4784.0,5.580488,200101,2001,01,5910822.0,11.0,"(38.1156440131474, -120.2655120173615)",NaN,...,38.115644,-120.265512,0.032258,155.088272,0.544187,2001.0,1.0,8.99,1017.0,89010
2676397,89019,4821.0,5.101090,200101,2001,01,5879815.0,12.0,"(32.79739819490558, -116.734579970192)",NaN,...,32.797398,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89019
2676398,89019,4821.0,5.101090,200101,2001,01,5910823.0,12.0,"(38.1156440131474, -120.2655120173615)",NaN,...,38.115644,-120.265512,0.032258,155.088272,0.544187,2001.0,1.0,8.99,1017.0,89019
2676399,89060,2621.0,4.714583,200101,2001,01,5879816.0,13.0,"(32.79739819490558, -116.734579970192)",NaN,...,32.797398,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006046,96150,6548.0,1.315252,200112,2001,12,5626852.0,1641.0,"(36.959895077007, -118.6981789253998)",NaN,...,36.959895,-118.698179,0.548387,22.306015,0.657902,2001.0,12.0,0.10,2569.0,96150
3006052,96150,6548.0,1.315252,200112,2001,12,5635012.0,1641.0,"(36.93386873267413, -118.6577934353378)",NaN,...,36.933869,-118.657793,0.032258,22.306015,0.657902,2001.0,12.0,0.10,2783.0,96150
3006058,96161,6042.0,1.524918,200112,2001,12,5626853.0,1642.0,"(36.959895077007, -118.6981789253998)",NaN,...,36.959895,-118.698179,0.548387,22.306015,0.657902,2001.0,12.0,0.10,2569.0,96161
3006064,96161,6042.0,1.524918,200112,2001,12,5635013.0,1642.0,"(36.93386873267413, -118.6577934353378)",NaN,...,36.933869,-118.657793,0.032258,22.306015,0.657902,2001.0,12.0,0.10,2783.0,96161


In [204]:
temp_temp["elevation_difference"] = (
    temp_temp["fire_elevation"] - temp_temp["zip_elevation"]
)

In [205]:
temp_temp[temp_temp['year_x']=="2001"]

,ZCTA_x,zip_elevation,zip_pm25,year_month,year_x,month_x,Unnamed: 0,ZCTA_y,fire_lat_lon,instrument,...,fire_lon,fire_days_in_mo,fire_wdir,fire_wspd_y,year_y,month_y,fire_pm25,fire_elevation,real_ZCTA,elevation_difference
2676395,89010,4784.0,5.580488,200101,2001,01,5879814.0,11.0,"(32.79739819490558, -116.734579970192)",NaN,...,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89010,-4256.0
2676396,89010,4784.0,5.580488,200101,2001,01,5910822.0,11.0,"(38.1156440131474, -120.2655120173615)",NaN,...,-120.265512,0.032258,155.088272,0.544187,2001.0,1.0,8.99,1017.0,89010,-3767.0
2676397,89019,4821.0,5.101090,200101,2001,01,5879815.0,12.0,"(32.79739819490558, -116.734579970192)",NaN,...,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89019,-4293.0
2676398,89019,4821.0,5.101090,200101,2001,01,5910823.0,12.0,"(38.1156440131474, -120.2655120173615)",NaN,...,-120.265512,0.032258,155.088272,0.544187,2001.0,1.0,8.99,1017.0,89019,-3804.0
2676399,89060,2621.0,4.714583,200101,2001,01,5879816.0,13.0,"(32.79739819490558, -116.734579970192)",NaN,...,-116.734580,0.193548,329.901306,0.363994,2001.0,1.0,4.74,528.0,89060,-2093.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006046,96150,6548.0,1.315252,200112,2001,12,5626852.0,1641.0,"(36.959895077007, -118.6981789253998)",NaN,...,-118.698179,0.548387,22.306015,0.657902,2001.0,12.0,0.10,2569.0,96150,-3979.0
3006052,96150,6548.0,1.315252,200112,2001,12,5635012.0,1641.0,"(36.93386873267413, -118.6577934353378)",NaN,...,-118.657793,0.032258,22.306015,0.657902,2001.0,12.0,0.10,2783.0,96150,-3765.0
3006058,96161,6042.0,1.524918,200112,2001,12,5626853.0,1642.0,"(36.959895077007, -118.6981789253998)",NaN,...,-118.698179,0.548387,22.306015,0.657902,2001.0,12.0,0.10,2569.0,96161,-3473.0
3006064,96161,6042.0,1.524918,200112,2001,12,5635013.0,1642.0,"(36.93386873267413, -118.6577934353378)",NaN,...,-118.657793,0.032258,22.306015,0.657902,2001.0,12.0,0.10,2783.0,96161,-3259.0


In [208]:
last_temp = temp_temp[['ZCTA_x', 'zip_elevation', 'zip_pm25', 'year_month', 'year_x', 'month_x', 'fire_lat_lon', 'instrument', 'bearing', 'dist',
       'fire_wspd', 'acres', 'fire_lat', 'fire_lon', 'fire_days_in_mo',
       'fire_wdir','fire_pm25',
       'fire_elevation', 'elevation_difference']]
temp_temp = last_temp.rename(columns={'ZCTA_x':'ZCTA','year_x':'year','month_x':'month'})           

In [212]:
temp_temp[temp_temp.year=="2001"]

,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,dist,fire_wspd,acres,fire_lat,fire_lon,fire_days_in_mo,fire_wdir,fire_pm25,fire_elevation,elevation_difference
2676395,89010,4784.0,5.580488,200101,2001,01,"(32.79739819490558, -116.734579970192)",NaN,0.914311,329.578834,0.363994,10438.019531,32.797398,-116.734580,0.193548,329.901306,4.74,528.0,-4256.0
2676396,89010,4784.0,5.580488,200101,2001,01,"(38.1156440131474, -120.2655120173615)",NaN,0.878819,134.909114,0.544187,9.962753,38.115644,-120.265512,0.032258,155.088272,8.99,1017.0,-3767.0
2676397,89019,4821.0,5.101090,200101,2001,01,"(32.79739819490558, -116.734579970192)",NaN,0.675674,181.101744,0.363994,10438.019531,32.797398,-116.734580,0.193548,329.901306,4.74,528.0,-4293.0
2676398,89019,4821.0,5.101090,200101,2001,01,"(38.1156440131474, -120.2655120173615)",NaN,0.833183,288.001608,0.544187,9.962753,38.115644,-120.265512,0.032258,155.088272,8.99,1017.0,-3804.0
2676399,89060,2621.0,4.714583,200101,2001,01,"(32.79739819490558, -116.734579970192)",NaN,0.746549,234.877790,0.363994,10438.019531,32.797398,-116.734580,0.193548,329.901306,4.74,528.0,-2093.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006046,96150,6548.0,1.315252,200112,2001,12,"(36.959895077007, -118.6981789253998)",NaN,0.919454,142.467234,0.657902,2213.307129,36.959895,-118.698179,0.548387,22.306015,0.10,2569.0,-3979.0
3006052,96150,6548.0,1.315252,200112,2001,12,"(36.93386873267413, -118.6577934353378)",NaN,0.919844,145.149119,0.657902,9.361344,36.933869,-118.657793,0.032258,22.306015,0.10,2783.0,-3765.0
3006058,96161,6042.0,1.524918,200112,2001,12,"(36.959895077007, -118.6981789253998)",NaN,0.513330,180.160974,0.657902,2213.307129,36.959895,-118.698179,0.548387,22.306015,0.10,2569.0,-3473.0
3006064,96161,6042.0,1.524918,200112,2001,12,"(36.93386873267413, -118.6577934353378)",NaN,0.510419,182.852422,0.657902,9.361344,36.933869,-118.657793,0.032258,22.306015,0.10,2783.0,-3259.0


## Adding normalization (min-max) for wspd, distance, acres, bearing

In [213]:
temp_max = np.max(temp_temp["fire_wspd"])
temp_min = np.min(temp_temp["fire_wspd"])
diff = temp_max - temp_min

temp_temp["fire_wspd_norm"] = temp_temp["fire_wspd"].map(
    lambda x: ((x - temp_min) / (diff))
)
print("Windspeed normalized")

temp_max = np.max(temp_temp["dist"])
temp_min = np.min(temp_temp["dist"])
diff = temp_max - temp_min

temp_temp["dist_norm"] = temp_temp["dist"].map(lambda x: ((x - temp_min) / (diff)))
print("Distance normalized")

temp_max = np.max(temp_temp["acres"])
temp_min = np.min(temp_temp["acres"])
diff = temp_max - temp_min

temp_temp["acres_norm"] = temp_temp["acres"].map(lambda x: ((x - temp_min) / (diff)))
print("Acres normalized")

temp_max = 1
temp_min = 0.5
diff = temp_max - temp_min

temp_temp["bearing_norm"] = temp_temp["bearing"].map(
    lambda x: ((x - temp_min) / (diff))
)
print("Bearings normalized")

Windspeed normalized
Distance normalized
Acres normalized
Bearings normalized


In [214]:
temp_temp["bearing_bins"] = temp_temp["bearing"].map(
    lambda x: 10
    if x > 0.95
    else (
        9
        if x > 0.9
        else (
            8
            if x > 0.85
            else (
                7
                if x > 0.8
                else (
                    6
                    if x > 0.75
                    else (
                        5
                        if x > 0.7
                        else (
                            4
                            if x > 0.65
                            else (3 if x > 0.6 else (2 if x > 0.55 else (1)))
                        )
                    )
                )
            )
        )
    )
)

In [215]:
temp_temp.drop_duplicates()

,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,dist,...,fire_days_in_mo,fire_wdir,fire_pm25,fire_elevation,elevation_difference,fire_wspd_norm,dist_norm,acres_norm,bearing_norm,bearing_bins
0,89010,4784.0,12.450976,199101,1991,01,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,287.182949,...,0.064516,274.314545,0.10,1320.0,-3464.0,0.166263,0.348253,0.000142,0.565210,6
1,89010,4784.0,12.450976,199101,1991,01,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,220.403951,...,0.032258,186.630615,11.24,1263.0,-3521.0,0.091785,0.267263,0.000055,0.876054,9
2,89010,4784.0,12.450976,199101,1991,01,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,283.790088,...,0.064516,274.314545,1.74,1013.0,-3771.0,0.166263,0.344138,0.000155,0.580796,6
3,89010,4784.0,12.450976,199101,1991,01,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,242.222915,...,0.096774,182.438004,10.59,1057.0,-3727.0,0.023638,0.293725,0.000100,0.993251,10
4,89010,4784.0,12.450976,199101,1991,01,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,240.497549,...,0.096774,186.630615,15.02,852.0,-3932.0,0.091785,0.291632,0.000266,0.988867,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19445497,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19445498,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19445499,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19445500,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [216]:
temp_temp["ins_1_no_bin_raw"] = (
    temp_temp["bearing"] * temp_temp["fire_wspd"] / temp_temp["dist"]
)
temp_temp["ins_2_add_acres_raw"] = (
    temp_temp["bearing"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)
temp_temp["ins_3_norms"] = (
    temp_temp["bearing_norm"] * temp_temp["fire_wspd_norm"] / temp_temp["dist_norm"]
)
temp_temp["ins_4_add_acres_norms"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_5_norm_dist_numerator"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_6_sub_bins_raw"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    * temp_temp["dist"]
)

In [217]:
temp_temp.drop_duplicates()

,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,dist,...,dist_norm,acres_norm,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw
0,89010,4784.0,12.450976,199101,1991,01,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,287.182949,...,0.348253,0.000142,0.565210,6,0.002533,0.113662,0.269844,0.000038,0.000038,71868.672976
1,89010,4784.0,12.450976,199101,1991,01,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,220.403951,...,0.267263,0.000055,0.876054,9,0.002197,0.038422,0.300859,0.000017,0.000017,17908.026722
2,89010,4784.0,12.450976,199101,1991,01,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,283.790088,...,0.344138,0.000155,0.580796,6,0.002588,0.126796,0.280600,0.000044,0.000044,77518.253168
3,89010,4784.0,12.450976,199101,1991,01,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,242.222915,...,0.293725,0.000100,0.993251,10,0.000568,0.017893,0.079933,0.000008,0.000008,10533.561275
4,89010,4784.0,12.450976,199101,1991,01,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,240.497549,...,0.291632,0.000266,0.988867,10,0.002134,0.179142,0.311224,0.000083,0.000083,104193.925266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19445497,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
19445498,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
19445499,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
19445500,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
temp_temp.to_csv(in_instrument + "new_non_agg.csv")

## Aggregation of the dataframe for OLS

In [219]:
instrument_df = pd.read_csv(in_instrument + "new_non_agg.csv")
instrument_df

,Unnamed: 0,ZCTA,zip_elevation,zip_pm25,year_month,year,month,fire_lat_lon,instrument,bearing,...,dist_norm,acres_norm,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw
0,0,89010,4784.0,12.450976,199101,1991,1,"(39.15535115700015, -122.7759650805785)",0.000163,0.782605,...,0.348253,0.000142,0.565210,6,0.002533,0.113662,0.269844,0.000038,0.000038,71868.672976
1,1,89010,4784.0,12.450976,199101,1991,1,"(39.56756030998135, -121.0023355975935)",0.000071,0.938027,...,0.267263,0.000055,0.876054,9,0.002197,0.038422,0.300859,0.000017,0.000017,17908.026722
2,2,89010,4784.0,12.450976,199101,1991,1,"(39.08499688682003, -122.7444818242734)",0.000167,0.790398,...,0.344138,0.000155,0.580796,6,0.002588,0.126796,0.280600,0.000044,0.000044,77518.253168
3,3,89010,4784.0,12.450976,199101,1991,1,"(39.75680610240665, -121.3315226354367)",0.000055,0.996625,...,0.293725,0.000100,0.993251,10,0.000568,0.017893,0.079933,0.000008,0.000008,10533.561275
4,4,89010,4784.0,12.450976,199101,1991,1,"(39.69440333991243, -121.355888160909)",0.000207,0.994434,...,0.291632,0.000266,0.988867,10,0.002134,0.179142,0.311224,0.000083,0.000083,104193.925266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10948125,19445497,96146,6649.0,4.626316,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
10948126,19445498,96148,6473.0,4.528572,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
10948127,19445499,96150,6548.0,4.286667,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
10948128,19445500,96161,6042.0,5.053509,201812,2018,12,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
instrument_df.groupby('year').sum()

,Unnamed: 0,ZCTA,zip_elevation,zip_pm25,year_month,month,instrument,bearing,dist,fire_wspd,...,dist_norm,acres_norm,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw
year,,,,,,,,,,,,,,,,,,,,,
1991,7489378910,9023318754,1.237734e+08,1.535037e+06,19183380092,692392,49.820869,70273.032029,2.423757e+07,55526.909084,...,29391.327961,119.135028,47465.064058,524057,273.769063,1.170650e+05,28556.186083,40.542400,40.542400,2.530290e+10
1992,42582400307,13152634116,1.804155e+08,1.851695e+06,27976309417,1059817,81.761095,103442.125146,3.606490e+07,101257.601285,...,43733.692928,516.476362,71345.250292,785756,491.628862,3.506703e+05,52285.086927,116.322391,116.322391,8.846258e+10
1993,69036090726,11929133268,1.636327e+08,1.708313e+06,25386590239,952039,55.116177,90384.732442,2.617726e+07,117665.656357,...,31742.559464,854.074767,59927.464883,666534,619.136076,1.604430e+06,63215.354664,549.786632,549.786632,3.414150e+11
1994,169853555600,19423075962,2.664276e+08,2.646882e+06,41355292255,1526855,147.626442,151005.281398,5.426137e+07,182434.760042,...,65799.579233,1495.653958,101151.562796,1117991,1027.280285,2.917739e+06,107863.707449,1005.497417,1005.497417,7.104997e+11
1995,179342756933,15446698206,2.118834e+08,2.021534e+06,32905454597,1321097,84.236362,120183.983641,3.795684e+07,165763.710008,...,46027.237614,452.950135,81966.967283,905143,985.806171,1.163167e+06,115858.986210,391.496691,391.496691,2.860679e+11
1996,426901551184,27681706686,3.797118e+08,3.568586e+06,58998617808,2247008,119.490441,221007.071259,7.352324e+07,240763.219295,...,89156.500443,2653.412000,151340.142518,1662649,1434.234032,4.237743e+06,154712.682438,1432.391173,1432.391173,1.030054e+12
1997,259600660359,13458509328,1.846112e+08,1.592005e+06,28698689829,1001029,109.815052,103835.314132,3.199289e+07,172765.812454,...,38795.064504,642.193843,70498.628264,779854,1001.653078,1.692658e+06,107265.978615,593.423177,593.423177,2.988554e+11
1998,161869279710,7799817906,1.069906e+08,9.462028e+05,16640577004,633604,20.074834,55663.127522,1.758357e+07,78235.271989,...,21322.190273,170.581036,37841.255045,424716,465.645159,1.304618e+05,49087.123330,42.163275,42.163275,4.210285e+10
1999,618703203233,25999393020,3.566354e+08,3.320371e+06,55496557860,2318860,145.608557,213013.686325,7.814054e+07,202429.138464,...,94756.970523,6638.899726,148417.372650,1621678,1206.932185,5.685489e+06,131167.678559,2011.065668,2011.065668,1.477842e+12


# START HERE FOR AGGREGATION

In [221]:
temp_temp = pd.read_csv(in_instrument + "new_non_agg.csv")

In [222]:
temp_temp.year.unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018])

In [223]:
wind_ready = pd.read_csv(in_dir + "wind_zip_pm25.csv")
wind_ready

,Unnamed: 0,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year,month,zip_pm25
0,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976
1,1,35.396,-116.322,89019,259.661102,0.962568,199101,1991,1,10.846541
2,2,36.161,-116.139,89060,241.796738,0.922477,199101,1991,1,12.385000
3,3,35.957,-115.897,89061,244.522552,1.303105,199101,1991,1,10.150000
4,4,39.520,-120.032,89439,78.390099,0.209957,199101,1991,1,8.389565
...,...,...,...,...,...,...,...,...,...,...
548683,548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316
548684,548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572
548685,548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667
548686,548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509


In [224]:
#Revision 4
temp_temp["ins_1_no_bin_raw"] = (
    temp_temp["bearing_norm"] 
    * temp_temp["fire_wspd"] 
    / temp_temp["dist"]
)
temp_temp["ins_2_add_acres_raw"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)
temp_temp["ins_3_norms"] = (
    temp_temp["bearing_norm"] 
    * temp_temp["fire_wspd_norm"] 
    / temp_temp["dist_norm"]
)
temp_temp["ins_4_add_acres_norms"] = (
    temp_temp["bearing_norm"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_5_norm_bins_acres"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd_norm"]
    * temp_temp["acres_norm"]
    / temp_temp["dist_norm"]
)
temp_temp["ins_6_bins_raw"] = (
    temp_temp["bearing_bins"]
    * temp_temp["fire_wspd"]
    * temp_temp["acres"]
    / temp_temp["dist"]
)

In [225]:
temp_temp["year_month"] = temp_temp["year_month"].astype(str)
temp_temp["ZCTA"] = temp_temp["ZCTA"].astype(str)
wind_ready["year_month"] = wind_ready["year_month"].astype(str)
wind_ready["ZCTA"] = wind_ready["ZCTA"].astype(str)


In [226]:
instrument_df = wind_ready.merge(temp_temp, on= ['ZCTA','year_month'],how='left').rename(columns={'wspd':'zip_wspd',
                                                                                                 'zip_pm25_y':
                                                                                                'zip_pm25'})
instrument_df

,Unnamed: 0_x,zip_lat,zip_lon,ZCTA,zip_wdir,zip_wspd,year_month,year_x,month_x,zip_pm25_x,...,bearing_norm,bearing_bins,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_dist_numerator,ins_6_sub_bins_raw,ins_5_norm_bins_acres,ins_6_bins_raw
0,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.565210,6,0.001829,0.082088,0.269844,0.000038,0.000038,71868.672976,0.000407,0.871410
1,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.876054,9,0.002052,0.035884,0.300859,0.000017,0.000017,17908.026722,0.000171,0.368646
2,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.580796,6,0.001902,0.093171,0.280600,0.000044,0.000044,77518.253168,0.000450,0.962519
3,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.993251,10,0.000566,0.017832,0.079933,0.000008,0.000008,10533.561275,0.000080,0.179533
4,0,37.465,-117.936,89010,305.042938,0.878208,199101,1991,1,12.450976,...,0.988867,10,0.002122,0.178139,0.311224,0.000083,0.000083,104193.925266,0.000837,1.801445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10948125,548683,39.149,-120.248,96146,76.585587,0.437017,201812,2018,12,4.626316,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10948126,548684,39.236,-120.062,96148,76.585587,0.437017,201812,2018,12,4.528572,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10948127,548685,38.732,-120.033,96150,107.369316,0.231856,201812,2018,12,4.286667,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10948128,548686,39.184,-120.427,96161,151.071350,0.243283,201812,2018,12,5.053509,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [227]:
instrument_df.columns

Index(['Unnamed: 0_x', 'zip_lat', 'zip_lon', 'ZCTA', 'zip_wdir', 'zip_wspd',
       'year_month', 'year_x', 'month_x', 'zip_pm25_x', 'Unnamed: 0_y',
       'zip_elevation', 'zip_pm25', 'year_y', 'month_y', 'fire_lat_lon',
       'instrument', 'bearing', 'dist', 'fire_wspd', 'acres', 'fire_lat',
       'fire_lon', 'fire_days_in_mo', 'fire_wdir', 'fire_pm25',
       'fire_elevation', 'elevation_difference', 'fire_wspd_norm', 'dist_norm',
       'acres_norm', 'bearing_norm', 'bearing_bins', 'ins_1_no_bin_raw',
       'ins_2_add_acres_raw', 'ins_3_norms', 'ins_4_add_acres_norms',
       'ins_5_norm_dist_numerator', 'ins_6_sub_bins_raw',
       'ins_5_norm_bins_acres', 'ins_6_bins_raw'],
      dtype='object')

In [228]:
instrument_df['treatment'] = instrument_df.apply(lambda x: 1 if (x['bearing']>0.75 and x['dist'] <=200) else 0, axis=1)

In [229]:
agg_dict = {
    "ins_1_no_bin_raw": "sum",
    "ins_2_add_acres_raw": "sum",
    "ins_3_norms": "sum",
    "ins_4_add_acres_norms": "sum",
    "ins_5_norm_bins_acres": "sum",
    "ins_6_bins_raw": "sum",
    "zip_pm25": "first",
    "zip_elevation": "mean",
    "elevation_difference": "mean",
    "zip_wspd":"first",
    "treatment":"max",
}

In [230]:
model_df2 = (
    instrument_df.groupby(  # [['ZCTA','year_month','fire_lat_lon','instrument','instrument_2','instrument_2_norm']]
        ["ZCTA", "year_month"]
    )
    .aggregate(agg_dict)
    .drop_duplicates()
)

In [234]:
model_df2.drop_duplicates()

ins_1_no_bin_raw  ins_2_add_acres_raw  ins_3_norms  \
ZCTA  year_month                                                       
89010 199101              0.014313             2.171619     2.095820   
      199102              0.003401             1.473353     0.499474   
      199103              0.000000             0.000000     0.000000   
      199104              0.024555             4.317860     3.633868   
      199105              0.000000             0.000000     0.000000   
...                            ...                  ...          ...   
97635 201808              0.000000             0.000000     0.000000   
      201809              0.000000             0.000000     0.000000   
      201810              0.000000             0.000000     0.000000   
      201811              0.000000             0.000000     0.000000   
      201812              0.000000             0.000000     0.000000   

                  ins_4_add_acres_norms  ins_5_norm_bins_acres  \
ZCTA  year_month                                                 
89010 199101                   0.001013               0.011318   
      199102                   0.000688               0.007878   
      199103                   0.000000               0.000000   
      199104                   0.002025               0.024022   
      199105                   0.000000               0.000000   
...                                 ...                    ...   
97635 201808                   0.000000               0.000000   
      201809                   0.000000               0.000000   
      201810                   0.000000               0.000000   
      201811                   0.000000               0.000000   
      201812                   0.000000               0.000000   

                  ins_6_bins_raw   zip_pm25  zip_elevation  \
ZCTA  year_month                                             
89010 199101           24.261588  12.450976         4784.0   
      199102           16.875655  11.255854         4784.0   
      199103            0.000000   7.130000         4784.0   
      199104           51.209927   7.899268         4784.0   
      199105            0.000000   7.004146         4784.0   
...                          ...        ...            ...   
97635 201808            0.000000  18.950000         7370.0   
      201809            0.000000   4.200000         7370.0   
      201810            0.000000   3.453125         7370.0   
      201811            0.000000   3.415625         7370.0   
      201812            0.000000   3.393750         7370.0   

                  elevation_difference  zip_wspd  treatment  
ZCTA  year_month                                             
89010 199101              -3523.444444  0.878208          1  
      199102              -3412.000000  0.283772          0  
      199103                       NaN  1.129854          0  
      199104              -3777.333333  1.547416          1  
      199105                       NaN  1.658761          0  
...                                ...       ...        ...  
97635 201808                       NaN  0.775684          0  
      201809                       NaN  0.599688          0  
      201810                       NaN  0.094439          0  
      201811                       NaN  0.914967          0  
      201812                       NaN  1.319343          0  

[548332 rows x 11 columns]

In [233]:
model_df2.to_csv(in_instrument + 'aggregated_draft_5.csv')

In [ ]:
model_df2

instrument  instrument_norm    bearing     distance  \
ZCTA  year_month                                                        
89019 199104        0.000261         0.006595   0.910217   185.292714   
      199107        0.000712         0.016371   5.377310   921.508343   
      199108        0.031378         0.717908   8.021400  1264.961710   
      199109        0.011910         0.275090  15.282616  2500.767308   
      199110        0.018859         0.453440  12.722091  2185.207157   
...                      ...              ...        ...          ...   
97635 201607        0.002553         0.036612  10.788656  1845.938462   
      201608        0.010405         0.152898  12.858277  2378.383437   
      201609        0.019936         0.275592  18.298730  3080.280585   
      201610        0.063282         1.022041   4.274471   443.813040   
      201611        0.011195         0.219072   2.608769   344.197776   

                  fire_wspd_x         acres    fire_lat     fire_lon  \
ZCTA  year_month                                                       
89019 199104         1.593638    207.908157   37.368992  -118.578197   
      199107         2.220699    733.828024  219.863048  -711.415978   
      199108         8.900734   3778.423931  325.520290 -1064.638511   
      199109         4.960312   7088.643047  617.857124 -2014.177914   
      199110         3.562893   6874.610214  510.727792 -1661.366877   
...                       ...           ...         ...          ...   
97635 201607         6.050591   1481.088247  529.755001 -1587.936493   
      201608         9.851180   3532.768287  611.579388 -1837.366152   
      201609        10.709470  12670.919912  894.747648 -2682.674963   
      201610         7.398554   8481.111023  204.069248  -604.084705   
      201611         1.582546   7721.865967  121.127814  -362.253242   

                  fire_days_in_mo    fire_wdir  fire_wspd_y   year  month  \
ZCTA  year_month                                                            
89019 199104             0.033333   356.694611     1.593638   1991      4   
      199107             0.354839   539.071667     2.220699  11946     42   
      199108             6.161290   434.880981     8.900734  17919     72   
      199109             9.066667  3306.518982     4.960312  33847    153   
      199110            13.096774  3704.847351     3.562893  27874    140   
...                           ...          ...          ...    ...    ...   
97635 201607             1.096774  1940.639033     6.050591  26208     91   
      201608             3.096774  3243.913956     9.851180  30240    120   
      201609             5.666667  5492.342896    10.709470  44352    198   
      201610             4.096774   277.967331     7.398554  10080     50   
      201611             2.800000   214.686996     1.582546   6048     33   

                  fire_pm25  fire_elevation  fire_pm25_norm  instrument_2  \
ZCTA  year_month                                                            
89019 199104          10.71          1630.0        0.121085      0.002795   
      199107          25.43         14253.0        0.287507      0.003166   
      199108          12.88         22620.0        0.145619      0.064362   
      199109          70.09         41888.0        0.792425      0.048156   
      199110          64.56         34152.0        0.729904      0.084305   
...                     ...             ...             ...           ...   
97635 201607          71.50          7937.0        0.808366      0.013238   
      201608          68.91          9088.0        0.779084      0.069987   
      201609         113.96         16837.0        1.288412      0.078531   
      201610          14.69          7816.0        0.166083      0.195177   
      201611          10.83          4938.0        0.122442      0.040414   

                  instrument_2_norm  
ZCTA  year_month                     
89019 199104               0.000293  
      199107 

In [ ]:
model_df2.to_csv(in_instrument + "aggregated_instrument_cols_useful_only.csv")

In [ ]:
model_df = fire_4d.drop(columns=["lat", "lon", "u", "v", "wind_lat_lon"])
model_df = model_df.drop(model_df[model_df["year"] > 2019].index)
model_df = model_df.drop(columns=["year", "month"])
model_df["fire_index"] = model_df["fire_index"].fillna(-1)
model_df["radius_mi"] = model_df["radius_mi"].fillna(-1)
model_df["treatment"] = model_df["treatment"].fillna(False)
model_df.fire_index = model_df.fire_index.astype(int)
model_df.fire_index = model_df.fire_index.astype(str)
model_df = model_df.drop_duplicates()
model_df["real_interaction"] = np.nan
model_df["real_distance"] = np.nan
model_df["interaction"] = np.nan
model_df["distance"] = np.nan
model_df["duration_days"] = model_df["duration_days"].fillna(-1)
model_df.groupby(["ZCTA", "year_month", "fire_index"]).first()
model_df